In [1]:
# @formatter:off
import random

import numpy as np
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
# @formatter:on
import os

os.chdir('../')
print(os.getcwd())

DATA="/data/smaite/smaite"

/home/viktor/workspace/smaite-scripts


In [2]:
#@formatter:off
files = !ls $DATA/*csv.jsonl
#@formatter:on
files = [f for f in files if 'processed-augmented' in f]
assert len(files) == 8, len(files)

In [4]:
import json
from tqdm.notebook import tqdm
from unidecode import unidecode




def load_json(d):
    try:
        return json.loads(d, strict=False)
    except Exception as e:
        print("FEK")
        print(d)
        print(unidecode(d))
        return json.loads(unidecode(d), strict=False)


def load_jsonl(path: str):
    print("loading", path)
    with open(path) as f:
        return [load_json(d) for d in tqdm(f.readlines())]


datasets = [load_jsonl(p) for p in files]

loading /data/smaite/smaite/processed-augmented-bbc.csv.jsonl


  0%|          | 0/246 [00:00<?, ?it/s]

loading /data/smaite/smaite/processed-augmented-climatefeedback.csv.jsonl


  0%|          | 0/136 [00:00<?, ?it/s]

loading /data/smaite/smaite/processed-augmented-factcheck.csv.jsonl


...

  0%|          | 0/5400 [00:00<?, ?it/s]

loading /data/smaite/smaite/processed-augmented-snopes.csv.jsonl


  0%|          | 0/4499 [00:00<?, ?it/s]

loading /data/smaite/smaite/processed-augmented-washingtonpost.csv.jsonl


  0%|          | 0/1416 [00:00<?, ?it/s]

In [5]:
datasets[1][0].keys()

dict_keys(['text', 'text_article', 'label', 'explanation'])

In [4]:
from handystuff.loaders import write_jsonl


def create_classification_ds(ds, f, skip_unknown=True, claim_only=False):
    f = f.replace('augmented', 'classification' if not claim_only else 'classification-claim-only')

    cds = [
        {"sentence": f"{d['text']}\n{d['text_article']}" if not claim_only else f"{d['text']}",
         "label": d['label']} for d in ds if d['label']
    ]
    if skip_unknown:
        cds = [d for d in cds if d['label'] != 'UNKNOWN']
    write_jsonl(cds, f)

for ds, f in zip(datasets, files):
    if all(n not in f for n in ['fullfact', 'bbc']):
        create_classification_ds(ds, f)

#@formatter:off
!cd data && cat processed-classification-factcheck.csv.jsonl processed-classification-snopes.csv.jsonl processed-classification-washingtonpost.csv.jsonl > combined-classification-factcheck-ds.jsonl
#@formatter:on

In [6]:
def create_explanation_ds(ds, f):
    f = f.replace('augmented', 'explanation')
    eds = [
        {"input": f"{d['text']}\n{d['text_article']}",
         "target": d['explanation']} for d in ds
    ]
    write_jsonl(eds, f)

for ds, f in zip(datasets, files):
    if any(n in f for n in ['fullfact', 'bbc', 'factcheck']):
        create_explanation_ds(ds, f)

#@formatter:off
!cd data/ && cat processed-explanation-bbc.csv.jsonl processed-explanation-factcheck.csv.jsonl processed-explanation-fullfact.csv.jsonl > combined-explanation-factcheck-ds.jsonl
#@formatter:on


In [5]:
for ds, f in zip(datasets, files):
    if all(n not in f for n in ['fullfact', 'bbc']):
        create_classification_ds(ds, f, claim_only=True)

#@formatter:off
!cd data && cat processed-classification-claim-only-factcheck.csv.jsonl processed-classification-claim-only-snopes.csv.jsonl processed-classification-claim-only-washingtonpost.csv.jsonl > combined-classification-claim-only-factcheck-ds.jsonl
#@formatter:on
